In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.linear_model import SGDClassifier as SGDC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from datetime import datetime
%matplotlib inline

Читаем входные файлы с данными

In [2]:
transactions = pd.read_csv('./datasets/transactions/transactions.csv')

In [4]:
transactions = pd.read_csv('./datasets/transactions/transactions.csv')
customers_gender = pd.read_csv('./datasets/transactions/customers_gender_train.csv')

Для каждого клиента подсчитываем различные mcc_code

In [3]:
customers_gender = customers_gender.sort_values(by="customer_id")

In [4]:
customers_ids = customers_gender["customer_id"].values

In [5]:
customers_ids

array([    6815,    22899,    27914, ..., 99988578, 99991245, 99999680])

In [7]:
transactions.columns

Index(['customer_id', 'tr_datetime', 'mcc_code', 'tr_type', 'amount',
       'term_id'],
      dtype='object')

In [28]:
transactions[:1]["tr_datetime"][0].split()[1]

'10:23:26'

In [50]:
def transactions_to_numpy(transactions, bound=10):
    def row_transform(row, term_ids):
        print(row.tr_datetime)
        result = []
        result.append(row.tr_datetime.split()[0])
        dt = datetime.strptime(row.tr_datetime.split()[1], '%I:%M:%S')
        result.append(dt.hour * 60 + dt.minute)
        result.append(row.mcc_code)
        result.append(row.tr_type)
        result.append(row.amount)
        result.append(term_ids[row.term_id])
        print(result)
        return np.array(result, dtype=np.int64).reshape(1, -1)
    term_ids = LabelEncoder().fit_transform(transactions["term_id"].values)
    transactions_numpy = defaultdict(lambda: np.array([]))
    for i, row in enumerate(transactions.itertuples()):
        if i > bound:
            break
        print(transactions_numpy)
        print(transactions_numpy[row.customer_id])
        transactions_numpy[row.customer_id] = np.concatenate([transactions_numpy[row.customer_id], \
                                                              row_transform(row, term_ids)])
    return transactions_numpy

In [51]:
transactions_to_numpy(transactions, 10)

KeyboardInterrupt: 

In [ ]:
transformed_transactions = transactions_to_numpy(transactions)

In [55]:
LabelEncoder().fit_transform(transactions["term_id"].values[:1000])

TypeError: '>' not supported between instances of 'str' and 'float'

In [ ]:
day_tr = transactions["day"].values

In [52]:
transactions["term_id"].values

array([nan, nan, nan, ..., '10217113', 'RU570124', '372701'], dtype=object)

In [ ]:
train, test, y_train, y_test = train_test_split(transactions, y, train_size=0.7, random_state=13)

In [3]:
train = transactions.groupby('customer_id') \
                    .apply(lambda x: x[['mcc_code']].unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)

In [5]:
train

,742,1711,1731,1799,2741,3000,3351,3501,4111,4112,...,8299,8398,8641,8699,8999,9211,9222,9311,9399,9402
customer_id,,,,,,,,,,,,,,,,,,,,,
6815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
31385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0
49101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
customers_gender = customers_gender.set_index('customer_id')

In [6]:
customers_gender

,gender
customer_id,
75562265,0
10928546,1
69348468,1
84816985,1
61009479,0
74045822,0
27979606,1
54129921,0
23160845,0


In [5]:
Y_train = customers_gender.loc[X.index].gender
Y_train = Y_train.reset_index()
del Y_train['customer_id']
Y_train = Y_train.dropna(0)

In [6]:
X_train = X.reset_index()
X_train = X_train.loc[Y_train.index].set_index('customer_id')

Обучаемся на всех данных

In [7]:
clf = GradientBoostingClassifier(random_state=13)
clf.fit(X_train, Y_train.values[:, 0])

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=13, subsample=1.0, verbose=0,
              warm_start=False)

Предсказываем пол для тестовых данных и создаём файл с ответом

In [10]:
X_test = X.drop(customers_gender.index)
result = pd.DataFrame(X_test.index, columns=['customer_id'])
result['gender'] = clf.predict_proba(X_test)[:, 1]

In [11]:
result.to_csv('baseline_a.csv', index=False)

In [ ]:
result.append(row.tr_datetime.split()[0])
        dt = datetime.strptime(row.tr_datetime.split()[1], '%I:%M:%S')
        result.append(dt.hour * 60 + dt.minute)

In [11]:
transactions["hour"] = transactions["tr_datetime"].apply(lambda x: 24 * int(x.split()[0]) + 
                                         datetime.strptime(x.split()[1],
                                                           '%H:%M:%S').hour)

ValueError: second must be in 0..59

In [15]:
for transactions["tr_datetime"].apply(lambda x: x.split()[1][-3:])

0          :26
1          :29
2          :56
3          :54
4          :42
5          :49
6          :08
7          :57
8          :19
9          :31
10         :17
11         :02
12         :35
13         :51
14         :10
15         :22
16         :24
17         :03
18         :03
19         :48
20         :08
21         :30
22         :34
23         :03
24         :46
25         :00
26         :40
27         :20
28         :00
29         :00
          ... 
6849316    :37
6849317    :07
6849318    :55
6849319    :34
6849320    :37
6849321    :38
6849322    :44
6849323    :11
6849324    :51
6849325    :35
6849326    :21
6849327    :48
6849328    :03
6849329    :27
6849330    :25
6849331    :13
6849332    :26
6849333    :55
6849334    :53
6849335    :41
6849336    :49
6849337    :48
6849338    :52
6849339    :16
6849340    :04
6849341    :02
6849342    :60
6849343    :59
6849344    :53
6849345    :30
Name: tr_datetime, Length: 6849346, dtype: object

In [10]:
datetime.strptime('0:33:42', '%H:%M:%S').hour

0